In [41]:
#Client ID 3a3322d6152c475d8a2e3a822655a738
#Client Secret 0d8a2210e9b54b468bf2abd03bab06ea

In [42]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [43]:
import spotipy
import spotipy.util as util
from spotipy import oauth2

In [44]:
CLIENT_ID = "3a3322d6152c475d8a2e3a822655a738"
CLIENT_SECRET = "0d8a2210e9b54b468bf2abd03bab06ea"

In [45]:
token = oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

C:\Users\ROSHAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [46]:
playlist_id="0OSwcMA3Ewkse4rti2bNCG"
user_id="mzabdsv3lqwaf49fvuj153gef"

In [47]:
playlist=sp.user_playlist_tracks(user_id, playlist_id)["items"]
playlist

[{'added_at': '2020-05-01T10:50:52Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/mzabdsv3lqwaf49fvuj153gef'},
   'href': 'https://api.spotify.com/v1/users/mzabdsv3lqwaf49fvuj153gef',
   'id': 'mzabdsv3lqwaf49fvuj153gef',
   'type': 'user',
   'uri': 'spotify:user:mzabdsv3lqwaf49fvuj153gef'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/692VvGTch5OLXj4zEE6H3y'},
      'href': 'https://api.spotify.com/v1/artists/692VvGTch5OLXj4zEE6H3y',
      'id': '692VvGTch5OLXj4zEE6H3y',
      'name': 'Radio Moscow',
      'type': 'artist',
      'uri': 'spotify:artist:692VvGTch5OLXj4zEE6H3y'}],
    'available_markets': [],
    'external_urls': {'spotify': 'https://open.spotify.com/album/7BpKrz38kMmgSTLrRU17ZO'},
    'href': 'https://api.spotify.com/v1/albums/7BpKrz38kMmgSTLrRU17ZO',
    'id': '7BpKrz38kMmgSTLrRU17ZO',
    'images': [{'height': 64

In [48]:
import pandas as pd

In [49]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
tracks=get_playlist_tracks(user_id,playlist_id)

In [50]:
tracks


[{'added_at': '2020-05-01T10:50:52Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/mzabdsv3lqwaf49fvuj153gef'},
   'href': 'https://api.spotify.com/v1/users/mzabdsv3lqwaf49fvuj153gef',
   'id': 'mzabdsv3lqwaf49fvuj153gef',
   'type': 'user',
   'uri': 'spotify:user:mzabdsv3lqwaf49fvuj153gef'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/692VvGTch5OLXj4zEE6H3y'},
      'href': 'https://api.spotify.com/v1/artists/692VvGTch5OLXj4zEE6H3y',
      'id': '692VvGTch5OLXj4zEE6H3y',
      'name': 'Radio Moscow',
      'type': 'artist',
      'uri': 'spotify:artist:692VvGTch5OLXj4zEE6H3y'}],
    'available_markets': [],
    'external_urls': {'spotify': 'https://open.spotify.com/album/7BpKrz38kMmgSTLrRU17ZO'},
    'href': 'https://api.spotify.com/v1/albums/7BpKrz38kMmgSTLrRU17ZO',
    'id': '7BpKrz38kMmgSTLrRU17ZO',
    'images': [{'height': 64

In [51]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id",
                              "danceability","energy","key","loudness","mode", "speechiness","instrumentalness",
                              "liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    
    for track in tracks:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [52]:
new_df=analyze_playlist(user_id,playlist_id)

In [53]:
new_df.shape

(411, 16)

In [54]:
new_df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Radio Moscow,Brain Cycles,250 Miles,3drJvEvfMS5mnJ4JtmkM5S,0.467,0.422,9,-11.077,0,0.0703,0.169000,0.0960,0.588,76.818,292000,4
1,Milo Greene,Milo Greene,1957,08cXy6KUizaAelYXtcew3w,0.388,0.595,1,-9.799,1,0.0325,0.000056,0.1780,0.596,190.043,204213,4
2,The Smashing Pumpkins,Mellon Collie And The Infinite Sadness (Deluxe...,1979 - Remastered 2012,5QLHGv0DfpeXLNFo7SFEy1,0.767,0.787,3,-9.897,1,0.0331,0.583000,0.0513,0.964,126.879,266200,4
3,Lucifer,Lucifer I,Abracadabra,4tLRIAfjJmTFtro5fPyiAx,0.391,0.670,2,-7.096,1,0.0289,0.750000,0.0563,0.106,134.890,354266,4
4,Cigarettes After Sex,Affection,Affection,4eVNSHRazoHgfYGv3AYFyZ,0.415,0.425,7,-11.232,1,0.0250,0.908000,0.1050,0.248,89.839,311000,4


In [57]:
new_df.to_csv("Spotify_Playlist.csv" ,index=False)

In [ ]:
df.